# Install Detectron2 Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/Chapter3/faster_RCNN")

In [3]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8MB 26kB/s 
     |████████████████████████████████| 6.6MB 36.6MB/s 
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.5.0+cu101 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
     |████████████████████████████████| 276kB 14.8MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44091 sha256=e5bd4c6a6ba7e20b93da90227442e0dec61f17981a3f608b7b303b70d9f85e28
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAM

In [4]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

1.5.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!unzip  /content/gdrive/MyDrive/Chapter3/faster_RCNN/obj125.zip -d /content/gdrive/MyDrive/Chapter3/yolov4/darknet/data/multi_model/train

unzip:  cannot find or open /content/gdrive/MyDrive/Chapter3/faster_RCNN/obj125.zip, /content/gdrive/MyDrive/Chapter3/faster_RCNN/obj125.zip.zip or /content/gdrive/MyDrive/Chapter3/faster_RCNN/obj125.zip.ZIP.


In [ ]:
#unzipping and copy
!rm ./train150_multi/*.txt

In [5]:
!pip install fvcore

     |████████████████████████████████| 51kB 5.1MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210624-cp37-none-any.whl size=60611 sha256=9c20cb9c707bf326f68584ae28acd8471a65d2bddb9e5571b682ce44fbb6fc8b
  Stored in directory: /root/.cache/pip/wheels/6d/3d/8b/911fda7dc70c40197710f00c0e0b6d0d11c4d0bfb620567894
Successfully built fvcore


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger


# import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper
import copy

# Import and Register Custom Detectron2 Data

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_train6", {}, "/content/drive/MyDrive/Chapter3/faster_RCNN/multi_model_jsonfile/train140_multi.json", "/content/drive/MyDrive/Chapter3/faster_RCNN/train140_multi")
register_coco_instances("my_val6", {}, "/content/drive/MyDrive/Chapter3/faster_RCNN/multi_model_jsonfile/val_multi.json", "/content/drive/MyDrive/Chapter3/faster_RCNN/val")
# register_coco_instances("my_dataset_test5", {}, "./multi_model/test_corn_GS1_sunny_multi.json", "./test_corn_GS1_sunny")


In [ ]:
#visualize training data
my_train_metadata = MetadataCatalog.get("my_train6")
dataset_dicts = DatasetCatalog.get("my_train6")

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

# Train Custom Detectron2 Detector

In [ ]:
from detectron2.data import detection_utils as utils
# def custom_mapper(dataset_dict):
#     # Implement a mapper, similar to the default DatasetMapper, but with your own customizations
#     dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
#     image = utils.read_image(dataset_dict["file_name"], format="BGR")
#     transform_list = [T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
#                       T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
#                       T.RandomBrightness(0.9, 1.1) 
#                       ]
#     image, transforms = T.apply_transform_gens(transform_list, image)
#     dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

#     annos = [
#         utils.transform_instance_annotations(obj, transforms, image.shape[:2])
#         for obj in dataset_dict.pop("annotations")
#         if obj.get("iscrowd", 0) == 0
#     ]
#     instances = utils.annotations_to_instances(annos, image.shape[:2])
#     dataset_dict["instances"] = utils.filter_empty_instances(instances)
#     return dataset_dict


In [ ]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

cfg = get_cfg()
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

  # def build_train_loader(cls, cfg):
  #       return build_detection_train_loader(cfg, mapper=custom_mapper)

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
#from .detectron2.tools.train_net import Trainer
#from detectron2.engine import DefaultTrainer
# select from modelzoo here: https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines

from detectron2.config import get_cfg
#from detectron2.evaluation.coco_evaluation import COCOEvaluator
import os

# cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_train6",)
cfg.DATASETS.TEST = ("my_val6",)


cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.001


cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 30000 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (2000, 10000)
cfg.SOLVER.GAMMA = 0.05
cfg.SOLVER.CHECKPOINT_PERIOD = 5000



cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 #your number of classes + 1

cfg.TEST.EVAL_PERIOD = 1000




In [12]:
##needed while training
cfg.OUTPUT_DIR="/content/drive/MyDrive/Chapter3/faster_RCNN/output_train140_multi"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[06/24 01:32:23 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_68b088.pkl: 421MB [00:07, 54.0MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_p

[06/24 01:32:33 d2.engine.train_loop]: Starting training from iteration 0
[06/24 01:32:51 d2.utils.events]:  eta: 6:55:34  iter: 19  total_loss: 3.775  loss_cls: 1.129  loss_box_reg: 0.345  loss_rpn_cls: 1.993  loss_rpn_loc: 0.198  time: 0.8637  data_time: 0.5619  lr: 0.000020  max_mem: 2740M
[06/24 01:33:08 d2.utils.events]:  eta: 6:55:18  iter: 39  total_loss: 2.321  loss_cls: 0.995  loss_box_reg: 0.810  loss_rpn_cls: 0.152  loss_rpn_loc: 0.197  time: 0.8658  data_time: 0.5573  lr: 0.000040  max_mem: 2740M
[06/24 01:33:25 d2.utils.events]:  eta: 6:55:01  iter: 59  total_loss: 1.752  loss_cls: 0.741  loss_box_reg: 0.650  loss_rpn_cls: 0.122  loss_rpn_loc: 0.152  time: 0.8568  data_time: 0.5343  lr: 0.000060  max_mem: 2740M
[06/24 01:33:41 d2.utils.events]:  eta: 6:31:18  iter: 79  total_loss: 1.660  loss_cls: 0.672  loss_box_reg: 0.796  loss_rpn_cls: 0.058  loss_rpn_loc: 0.109  time: 0.8390  data_time: 0.4752  lr: 0.000080  max_mem: 2740M
[06/24 01:33:58 d2.utils.events]:  eta: 6:38:4

In [ ]:
##needed while evaluating on test data
cfg.MODEL.WEIGHTS = "./output/model_0019999.pth"
trainer = CocoTrainer(cfg)
trainer.resume_or_load()



In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output_multi

In [ ]:
##needed while evaluating on test data
##registering the test data
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
testname= "soybean3"
register_coco_instances(testname, {}, "./multi_model/test_soybean_GS3_cloudy_multi.json", "./test_soybean_GS3_cloudy")


In [ ]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset



cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator(testname, cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, testname)
inference_on_dataset(trainer.model, val_loader, evaluator)

# Inference with Detectron2 Saved Weights



In [ ]:
%ls ./output/

In [ ]:
# cfg.MODEL.WEIGHTS = os.path.join("/content/drive/MyDrive/Chapter3/faster_RCNN/output/model_final.pth")
# cfg.DATASETS.TEST = ("soy_multi2", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get(testname)
# test_metadata = MetadataCatalog.get("my_val").thing_classes=["MG", "Grass", "Other"]

# test_metadata = MetadataCatalog.get("my_train1").thing_classes = ["brown", "black"]


In [ ]:
test_metadata

Metadata(evaluator_type='coco', image_root='./test_soybean_GS3_cloudy', json_file='./multi_model/test_soybean_GS3_cloudy_multi.json', name='soybean3')

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob

imageName='/content/drive/MyDrive/Chapter3/faster_RCNN/test_soybean_GS3_cloudy/DSCF3649_1_2.JPG'
im = cv2.imread(imageName)
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
              metadata=test_metadata, 
              scale=0.8
                )
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
    pred_classes = outputs['instances'].pred_classes.cpu().tolist()
    values, counts = np.unique(pred_classes, return_counts=True)

In [ ]:
    import pandas as pd
    os.chdir("/content/drive/MyDrive/Chapter3/faster_RCNN/test_soybean_GS3_cloudy")
    df=pd.DataFrame(columns=["image_name", "MG", "Grass", "Other"])

    id=0
    for img in glob.glob("*.JPG")[0:5]:
      imagename=os.path.split(img)[1]
      im = cv2.imread(img)

      outputs = predictor(im)

      pred_classes = outputs['instances'].pred_classes.cpu().tolist()
      values, counts = np.unique(pred_classes, return_counts=True)
      if len(values.tolist())==3:
        MG=counts[0]
        Grass=counts[1]
        other=counts[2]

      if len(values.tolist())==2:
        if values.tolist()[0]==0:
          MG=counts[0]
        if values.tolist()[0]==1:
          Grass=counts[0]
          MG=0
        if values.tolist()[0]==2:
          Other=counts[0]
          MG=0
          Grass=0
        if values.tolist()[1]==1:
          Grass=counts[1]
          Other=0
        if values.tolist()[1]==2:
          Other=counts[1]
          Grass=0

      if len(values.tolist())==1:
        if values.tolist()[0]==0:
          MG=counts[0]
          Grass=0
          Other=0
        if values.tolist()[0]==1:
          Grass=counts[0]
          MG=0
          Other=0
        if values.tolist()[0]==2:
          Other=counts[0]
          MG=0
          Other=0

      df.loc[id]= [imagename] + [MG] + [Grass] + [Other] 
      print("completed predicting and counting for {} ". format (imagename))
      id+=1


df.to_excel("/content/drive/MyDrive/Chapter3/faster_RCNN/multi_model/soy2.xlsx")